In [ ]:
# Imports
from datetime import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### Loading Datasets

In [ ]:
# file path
file_path="C:/Users/Davie/Desktop/introduction-to-power-bi/"

In [ ]:
#load demographic data
demographic=pd.read_excel(file_path+"KPMG/KPMG_VI_New_raw_data_update_final.xlsx",sheet_name='CustomerDemographic', index_col=False, header=0, usecols="A:M", skiprows=1)

In [ ]:
#load customer address
address=pd.read_excel(file_path+"KPMG/KPMG_VI_New_raw_data_update_final.xlsx",sheet_name='CustomerAddress', index_col=False, header=0, usecols="A:F", skiprows=1)

In [ ]:
#load transaction data
transactions=pd.read_excel(file_path+"KPMG/KPMG_VI_New_raw_data_update_final.xlsx",sheet_name='Transactions', index_col=False, header=0, usecols="A:M", skiprows=1)

In [ ]:
#load transaction data
new_customers=pd.read_excel(file_path+"KPMG/KPMG_VI_New_raw_data_update_final.xlsx",sheet_name='NewCustomerList', index_col=False, header=0, usecols="A:P", skiprows=1)

### Merging the datasets

In [ ]:
#merge demographic data with customer address
demographic_address=pd.merge(demographic, address, on='customer_id', how='inner')

In [ ]:
#merged all the 3 datasets
demographic_address_transactions=pd.merge(demographic_address, transactions, on='customer_id', how='inner')

### Data Cleaning

In [ ]:
# Drop deceased persons
df=demographic_address_transactions[demographic_address_transactions['deceased_indicator']=='N']

In [ ]:
# Drop duplicated customer ids
data=df.dropna(how='any').drop_duplicates()

In [ ]:
new_customers.rename(columns={'past_3_years_bike_related_purchases': 'bikes_purchased'}, inplace=True)

In [ ]:
cus_data=new_customers[new_customers['deceased_indicator']=='N']

### Feature engineering

##### Target Customers

In [ ]:
# Calculate age

# convert DOB to datetime
cus_data['DOB']=pd.to_datetime(cus_data['DOB'], errors='coerce')
# Get the current date
current_date = pd.to_datetime('today')

# Now you can safely calculate age
cus_data['age'] = cus_data['DOB'].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)) if pd.notnull(x) else None)

In [ ]:
# Define age bins and labels
bins = [0, 20, 30, 40, 50, 60, 70, 100]
labels = ['<20','20-30','30-40','40-50','50-60','60-70', '>70']

# Create age groups
cus_data['age_group'] = pd.cut(cus_data['age'], bins=bins, labels=labels, right=False)

In [ ]:
# Define tenure ranges
bins = [0, 5, 10, 15, 25]
labels = ['<5','5-10', '10-15', '>15']

# Create age groups
cus_data['tenure_period'] = pd.cut(cus_data['tenure'], bins=bins, labels=labels, right=False)

In [ ]:
# Define proterty valuation ranges
bins = [0, 3, 6, 9, 13]
labels = ['<3','3-6', '6-9', '>9']

# Create age groups
cus_data['valuation_category'] = pd.cut(cus_data['property_valuation'], bins=bins, labels=labels, right=False)

In [ ]:
cus_data_col=['first_name', 'last_name', 'gender', 'bikes_purchased','DOB', 'age', 'age_group', 'state',
       'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'tenure', 'tenure_period','property_valuation', 'valuation_category']

In [ ]:
targeted_new_customers=cus_data[cus_data_col].dropna(how='any').drop_duplicates()

In [ ]:
targeted_new_customers.head()

##### Training dataset

In [ ]:
#calculate product margin
data['product_margin']=(data['list_price']-data['standard_cost'])/data['list_price']

In [ ]:

# Calculate age

# convert DOB to datetime
data['DOB']=pd.to_datetime(data['DOB'], errors='coerce')
# Get the current date
current_date = pd.to_datetime('today')

# Now you can safely calculate age
data['age'] = data['DOB'].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)) if pd.notnull(x) else None)

In [ ]:
# Define age bins and labels
bins = [0, 20, 30, 40, 50, 60, 70, 100]
labels = ['<20','20-30','30-40','40-50','50-60','60-70', '>70']

# Create age groups
data['age_group'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

In [ ]:
# Date of transaction
data['transaction_date'] = pd.to_datetime(data['transaction_date'])

# Extract the daya, monthand year from transaction_date
data['trans_day'] = data['transaction_date'].dt.day
data['trans_month'] = data['transaction_date'].dt.month

In [ ]:
# Replace values for gender and state in the entire DataFrame
data['gender'] = data['gender'].replace({'Femal': 'Female', 'F': 'Female'})
data['state']=data['state'].replace({'New South Wales':'NSW','Victoria':'VIC'})

In [ ]:
# Define tenure ranges
bins = [0, 5, 10, 15, 25]
labels = ['<5','5-10', '10-15', '>15']

# Create age groups
data['tenure_period'] = pd.cut(data['tenure'], bins=bins, labels=labels, right=False)

In [ ]:
# Define proterty valuation ranges
bins = [0, 3, 6, 9, 13]
labels = ['<3','3-6', '6-9', '>9']

# Create age groups
data['valuation_category'] = pd.cut(data['property_valuation'], bins=bins, labels=labels, right=False)

In [ ]:
data.rename(columns={'past_3_years_bike_related_purchases':'bikes_purchased'},inplace=True)

In [ ]:
# Distribution datasets
historical_data=data.copy().drop_duplicates(subset='customer_id').dropna(how='any')
target_data=targeted_new_customers.copy()

### Distribution for Bikes Purchased to be used as the target variable

In [ ]:
# Histogram

historical_data['bikes_purchased'].plot.hist(bins=50)

In [ ]:
mean=historical_data['bikes_purchased'].mean()
median=historical_data['bikes_purchased'].median()
mode=historical_data['bikes_purchased'].mode()
max=historical_data['bikes_purchased'].max()
min=historical_data['bikes_purchased'].min()
print(f'Summary Statistics: {mean}, {median}, {mode}, {min}, {max}')

In [ ]:
# histogram
targeted_new_customers['bikes_purchased'].plot.hist(bins=50)

In [ ]:
mean=target_data['bikes_purchased'].mean()
median=target_data['bikes_purchased'].median()
mode=target_data['bikes_purchased'].mode()
max=target_data['bikes_purchased'].max()
min=target_data['bikes_purchased'].min()
print(f'Summary Statistics: {mean}, {median}, {mode}, {min}, {max}')

### Columns

In [ ]:
# Check new columns for historical data
data.columns

In [ ]:
# Check new columns for targeted data
targeted_new_customers.columns

In [ ]:
# Select columns to use in the Model
cols=['first_name', 'last_name', 'gender', 'bikes_purchased','DOB', 'age', 'age_group', 'state',
       'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'tenure', 'tenure_period','property_valuation', 'valuation_category'] # should be the same both datasets used

#### Modelling Datasets

In [ ]:
# Make copy of the data
regression_model_data=data[cols].copy()
regression_model_data.reset_index(drop=True, inplace=True)
regression_model_data

In [ ]:
# Make copy of the data
regression_target_data=targeted_new_customers[cols].copy()
regression_model_data.reset_index(drop=True, inplace=True)
regression_target_data

### Encoding, Setting target and feature variables

In [ ]:
# Choose the encoding method based on the nature of your data and the requirements of your machine learning model. 
# One-hot encoding- is suitable when there is no ordinal relationship between categories
# label encoding- is useful when there is an ordinal relationship between categories. 
# Label encoding for Ordinal Variables
# ordinal_mapping_prod_size = {'small': 0, 'medium': 1, 'large': 2} 
# ordinal_data['product_size'] = ordinal_data['product_size'].map(ordinal_mapping_prod_size)

#Always remember to handle unknown categories appropriately, especially when using one-hot encoding.

In [ ]:
# I want to detertime customer demographic and product characteristics that influences bike purchase and identify 
# and target customers based on their demographic attributes
# So since only less tha 1% of the customer did not buy bikes, it makes this a regression problem

In [ ]:
# Define categorical columns and numerical columns
categorical_features=['gender', 'bikes_purchased','age_group', 'state',
       'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'tenure_period', 'valuation_category']

numerical_features = ['age', 'property_valuation', 'tenure'] 

# Initialize the LabelEncoder
label_encoders = {col: LabelEncoder() for col in categorical_features}

# Encode the regression model data columns
for col in categorical_features:
    regression_model_data[col] = label_encoders[col].fit_transform(regression_model_data[col])
regression_model_data.reset_index(drop=True, inplace=True)

# Encode the targeted new customer data columns
for col in categorical_features:
    regression_target_data[col] = label_encoders[col].fit_transform(regression_target_data[col])
regression_target_data.reset_index(drop=True, inplace=True)

### Preprocessing 

In [ ]:
# Preprocessing pipeline for numerical data
numerical_transformer = StandardScaler()

# Combine preprocessing steps for numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features)
    ],
    remainder='passthrough'  # Keep all other columns as they are.
)

### Define Target and Feature variables

In [ ]:
# Define the feature and the target variable for the historical data
features_X = regression_model_data.drop(columns=['bikes_purchased','first_name', 'last_name','DOB' ]) 
target_y = regression_model_data['bikes_purchased']

# Define the columns for customer target
X_new_customers = regression_target_data[features_X.columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_X, target_y, test_size=0.3, random_state=42)

### (a) Random Forest

In [ ]:
# Create a pipeline with LogisticRegression
model_ran = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [ ]:
# Train the model
model_ran.fit(X_train, y_train)

# Predict on the test set
y_pred = model_ran.predict(X_test)

# Evaluate the model
mae=mean_absolute_error(y_test,y_pred)
mse=mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False) # rmse=mse*0.5
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R_squraed (R_Squared): {r2}')

### (b) Decission Tree

In [ ]:
# Create a pipeline with DecisionTreeClassifier
model_det = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

In [ ]:
# Train the model
model_det.fit(X_train, y_train)

# Predict on the test set
y_pred = model_det.predict(X_test)

# Evaluate the model
mae=mean_absolute_error(y_test,y_pred)
mse=mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R_squraed (R_Squared): {r2}')

### (c) Linear Regression

In [ ]:
# We want to determine which demographic and product characteristics influences bike purchases or make customer to purchase the bike

In [ ]:
# Define the model pipeline
model_lin = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# Train the model
model_lin.fit(X_train, y_train)

# Predict on the test set
y_pred = model_lin.predict(X_test)

# Evaluate the model
mae=mean_absolute_error(y_test,y_pred)
mse=mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R_squraed (R_Squared): {r2}')

### Predictions base on the optimal model

In [ ]:
# Predictions on the new target customer dataset
new_customer_predictions = model_ran.predict(X_new_customers)

# Define a threshold for selecting customers
threshold = 49  # the mean number of bikes purchased in both datasets

# Select customers whose predicted values exceed the threshold
selected_customers = targeted_new_customers[new_customer_predictions > threshold]

# Attach predictions to the selected customer DataFrame
selected_customers = selected_customers.copy()  # to avoid SettingWithCopyWarning
selected_customers['predicted_value'] = new_customer_predictions[new_customer_predictions > threshold]

# Sort customers by their predicted values
target_customers=selected_customers.sort_values(by='predicted_value', ascending=False)

# Print target customers
target_customers

### Save Targeted Customers

In [ ]:
# Save the selected customers to a CSV file (optional)
target_customers.to_csv('selected_customers.csv', index=False)